In [6]:
import os
import cv2
import sys
import pandas as pd
from tqdm import tqdm
import math
sys.path.insert(0, '..')
from src import *

size = 256
fold = 0

root = os.path.join(os.environ["DATASET_ROOT"], "bcd2022")
destination = os.path.join(os.environ["DATASET_ROOT"], "bcd2022", f"patch_{size}_fold_{fold}", "0")
image_root = os.path.join(root, "images_1024")
df = pd.read_csv(os.path.join("..","data","train_5fold_aug.csv"))
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,...,machine_id,difficult_negative_case,fold,file,window_width,window_center,rows,cols,interpret,bits
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,...,29,False,0,10006_462822612.npz,1091.97,1802.31,5355,4915,MONOCHROME1,16
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,...,29,False,0,10006_1459541791.npz,1091.97,1802.31,5355,4915,MONOCHROME1,16
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,...,29,False,3,10006_1864590858.npz,1091.97,1802.31,5355,4915,MONOCHROME1,16
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,...,29,False,3,10006_1874946579.npz,1091.97,1802.31,5355,4915,MONOCHROME1,16
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,...,21,True,0,10011_220375232.npz,4096.00,2048.00,2776,2082,MONOCHROME2,12


In [7]:
def get_mil_instance_grid(image, num_wh, overlap_perc_wh, input_size=None):
    img_h, img_w = image.shape
    instance_w = math.floor(img_w / ((1 - overlap_perc_wh[0]) * (num_wh[0] - 1) + 1))
    instance_h = math.floor(img_h / ((1 - overlap_perc_wh[1]) * (num_wh[1] - 1) + 1))

    step_w = (1 - overlap_perc_wh[0]) * instance_w
    step_h = (1 - overlap_perc_wh[1]) * instance_h

    instance_list = []
    for i in range(num_wh[1]):
        i1 = int(i * step_h)
        i2 = int(i1 + instance_h)
        for j in range(num_wh[0]):
            j1 = int(j * step_w)
            j2 = int(j1 + instance_w)
            if input_size is not None:
                instance = cv2.resize(image[i1:i2, j1:j2], input_size)
            else:
                instance = image[i1:i2, j1:j2]

            instance_list.append(instance)

    return instance_list

In [8]:
df_train = df[df["fold"]!=0]
df_train["file"] = df_train["patient_id"].astype(str)+"_"+df_train["image_id"].astype(str)+".png"
files = [df_train[df_train["cancer"]==1].sample(800)["file"].values]
files += [df_train[df_train["cancer"]==0].sample(5000)["file"].values]
files = np.concatenate(files)
print(len(files))

5800


/tmp/ipykernel_4353/4183416950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["file"] = df_train["patient_id"].astype(str)+"_"+df_train["image_id"].astype(str)+".png"


In [9]:

nfile = len(files)

os.makedirs(destination, exist_ok=True)

index = 0
for k, f in enumerate(tqdm(files)):
    path = os.path.join(image_root, f)
    img = cv2.imread(path, 0)
    img = crop_breast(img)
    instances = get_mil_instance_grid(img, (1024//size, 1024//size), (0.15, 0.15), (size, size))
    for patch in instances:
        if patch.mean() > 25:
            cv2.imwrite(os.path.join(destination, f"{index:06d}.png"), patch)
        index += 1


100%|██████████| 5800/5800 [01:43<00:00, 55.99it/s]
